In [1]:
#Edgar Moises Hernandez-Gonzalez
#08/07/19-17/12/19
#Clasificar habla imaginada en EEG con CNN y LSTM, con CV=5

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Conv1D, MaxPool1D, Dropout, Flatten, LSTM, Dense

In [2]:
datos = pd.read_csv("Submuestreo01.csv", header=None)
x = datos.iloc[:, 0:3072]
y = datos.iloc[:, 3072]

In [3]:
print(x.shape)
print(y.shape)

(132, 3072)
(132,)


In [4]:
n_samples = len(y)
print("n_samples:", n_samples)

n_samples: 132


In [5]:
n_classes = len(np.unique(y))
print("n_classes:", n_classes)

n_classes: 6


In [6]:
#escalar los datos para que tengan media = 0 y desviacion estandar = 1
x = StandardScaler().fit_transform(x)

In [7]:
#convertir de 2D a 4D
x = x.reshape((n_samples, 1, 6, 512))
print(x.shape)

(132, 1, 6, 512)


In [8]:
x = np.transpose(x, (0,1,3,2))
print(x.shape)

(132, 1, 512, 6)


In [9]:
skf = StratifiedKFold(n_splits = 5)

In [11]:
resultados = np.zeros((2, 5)) #accuracy y loss en cada validacion cruzada
i = 0
for iTrain, jTest in skf.split(x, y):
    print("\nIteracion:", i + 1)
    x_train, x_test = x[iTrain], x[jTest]
    y_train, y_test = y[iTrain], y[jTest]
    
    model = Sequential()
    model.add(TimeDistributed(Conv1D(64, 3, activation='relu'), input_shape=(None,512,6)))
    model.add(TimeDistributed(Conv1D(64, 3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPool1D(2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_classes, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    model.fit(x_train, y_train, epochs = 20, batch_size = 64, validation_data=(x_test, y_test))
    
    resultados[0, i], resultados[1, i] = model.evaluate(x_test, y_test)
    i += 1


Iteracion: 1
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 103 samples, validate on 29 samples
Epoch 1/20
103/103 [==============================] - 8s 75ms/sample - loss: 1.8490 - acc: 0.1942 - val_loss: 1.7879 - val_acc: 0.2069
Epoch 2/20
103/103 [==============================] - 0s 1ms/sample - loss: 1.7324 - acc: 0.2233 - val_loss: 1.7990 - val_acc: 0.2069
Epoch 3/20
103/103 [==============================] - 0s 2ms/sample - loss: 1.7140 - acc: 0.2913 - val_loss: 1.8127 - val_acc: 0.2069
Epoch 4/20
103/103 [==============================] - 0s 2ms/sample - loss: 1.5515 - acc: 0.4757 - val_loss: 1.8147 - val_acc: 0.2069
Epoch 5/20
103/103 [==============================] - 0s 2ms/sample - loss: 1.3838 - acc: 0.5534 - val_loss: 1.8408 - val_acc: 0.1034
Epoch 6/20
103/103 [==============================] - 0s 2ms/sample - loss: 1.1857 

106/106 [==============================] - 0s 2ms/sample - loss: 0.4837 - acc: 0.9528 - val_loss: 2.0094 - val_acc: 0.2692
Epoch 15/20
106/106 [==============================] - 0s 2ms/sample - loss: 0.3553 - acc: 0.9811 - val_loss: 2.0371 - val_acc: 0.2692
Epoch 16/20
106/106 [==============================] - 0s 1ms/sample - loss: 0.3248 - acc: 0.9906 - val_loss: 2.0805 - val_acc: 0.3077
Epoch 17/20
106/106 [==============================] - 0s 1ms/sample - loss: 0.2734 - acc: 0.9906 - val_loss: 2.1328 - val_acc: 0.3077
Epoch 18/20
106/106 [==============================] - 0s 2ms/sample - loss: 0.2426 - acc: 1.0000 - val_loss: 2.1967 - val_acc: 0.2692
Epoch 19/20
106/106 [==============================] - 0s 1ms/sample - loss: 0.2518 - acc: 0.9717 - val_loss: 2.3133 - val_acc: 0.1923
Epoch 20/20
26/26 [==============================] - 0s 456us/sample - loss: 2.3871 - acc: 0.1923

Iteracion: 4
Train on 107 samples, validate on 25 samples
Epoch 1/20
107/107 [=========================

In [12]:
print("loss:", resultados[0])
print("accuracy:", resultados[1])
print("mean accuracy:", resultados[1].mean())

loss: [2.80984139 2.18751121 2.38714576 2.62151194 2.10788703]
accuracy: [0.1724138  0.33333334 0.1923077  0.16       0.31999999]
mean accuracy: 0.23561096489429473


In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_20 (TimeDis (None, None, 510, 64)     1216      
_________________________________________________________________
time_distributed_21 (TimeDis (None, None, 508, 64)     12352     
_________________________________________________________________
time_distributed_22 (TimeDis (None, None, 508, 64)     0         
_________________________________________________________________
time_distributed_23 (TimeDis (None, None, 254, 64)     0         
_________________________________________________________________
time_distributed_24 (TimeDis (None, None, 16256)       0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               6542800   
_________________________________________________________________
dropout_9 (Dropout)          (None, 100)               0         
__________